### Header

In [1]:
# import libraries

# maths
import numpy as np
import pandas as pd
import scipy.stats as stats
from pandas.api.types import is_numeric_dtype

# visual
#from matplotlib_venn import venn2
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# modelling
from sklearn import linear_model
from sklearn.linear_model import Ridge,LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.metrics import r2_score,mean_squared_error,confusion_matrix,accuracy_score
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.utils import resample

# nlp
from sklearn.feature_extraction.text import CountVectorizer,HashingVectorizer,TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import PorterStemmer,WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
from spacy.tokens import Doc

# web
import requests
import json

# others
import os
import re
import time
import datetime as datetime

In [2]:
# file paths

input_path = '../data/2_input/'
clean_path = '../data/3_clean/'
output_path = '../data/4_output/'

image_path = '../images/'

### Functions

### Import Data

In [3]:
# import clean data

df_train = pd.read_csv(clean_path + 'train_clean.csv')
df_test = pd.read_csv(clean_path + 'test_clean.csv')
df_weather = pd.read_csv(clean_path + 'weather_clean.csv')
#df_spray = pd.read_csv(clean_path + 'spray_clean.csv')

In [4]:
majority_class = df_train[df_train['wnvpresent']==0]
minority_class = df_train[df_train['wnvpresent']==1]

### Preprocessing

In [5]:
df_train.shape

(8475, 11)

In [6]:
df_test.shape

(116293, 10)

In [7]:
test = df_test.drop('id', axis=1)
train = df_train.drop(['nummosquitos', 'wnvpresent'], axis=1)
combined_train_test = pd.concat([test,train])
combined_train_test.shape

(124768, 9)

In [8]:
all_dataset = combined_train_test.merge(df_weather, how='left', on=['year','month','day'])
all_dataset.shape

(249536, 30)

In [11]:
all_dataset

species  block              street  trap  year  month  \
0       CULEX PIPIENS/RESTUANS     41      N OAK PARK AVE  T002  2008      6   
1       CULEX PIPIENS/RESTUANS     41      N OAK PARK AVE  T002  2008      6   
2               CULEX RESTUANS     41      N OAK PARK AVE  T002  2008      6   
3               CULEX RESTUANS     41      N OAK PARK AVE  T002  2008      6   
4                CULEX PIPIENS     41      N OAK PARK AVE  T002  2008      6   
5                CULEX PIPIENS     41      N OAK PARK AVE  T002  2008      6   
6             CULEX SALINARIUS     41      N OAK PARK AVE  T002  2008      6   
7             CULEX SALINARIUS     41      N OAK PARK AVE  T002  2008      6   
8              CULEX TERRITANS     41      N OAK PARK AVE  T002  2008      6   
9              CULEX TERRITANS     41      N OAK PARK AVE  T002  2008      6   
10              CULEX TARSALIS     41      N OAK PARK AVE  T002  2008      6   
11              CULEX TARSALIS     41      N OAK PARK AVE  T002  2008      6   
12           UNSPECIFIED CULEX     41      N OAK PARK AVE  T002  2008      6   
13           UNSPECIFIED CULEX     41      N OAK PARK AVE  T002  2008      6   
14             CULEX ERRATICUS     41      N OAK PARK AVE  T002  2008      6   
15             CULEX ERRATICUS     41      N OAK PARK AVE  T002  2008      6   
16      CULEX PIPIENS/RESTUANS     62       N MANDELL AVE  T007  2008      6   
17      CULEX PIPIENS/RESTUANS     62       N MANDELL AVE  T007  2008      6   
18              CULEX RESTUANS     62       N MANDELL AVE  T007  2008      6   
19              CULEX RESTUANS     62       N MANDELL AVE  T007  2008      6   
20               CULEX PIPIENS     62       N MANDELL AVE  T007  2008      6   
21               CULEX PIPIENS     62       N MANDELL AVE  T007  2008      6   
22            CULEX SALINARIUS     62       N MANDELL AVE  T007  2008      6   
23            CULEX SALINARIUS     62       N MANDELL AVE  T007  2008      6   
24             CULEX TERRITANS     62       N MANDELL AVE  T007  2008      6   
25             CULEX TERRITANS     62       N MANDELL AVE  T007  2008      6   
26              CULEX TARSALIS     62       N MANDELL AVE  T007  2008      6   
27              CULEX TARSALIS     62       N MANDELL AVE  T007  2008      6   
28           UNSPECIFIED CULEX     62       N MANDELL AVE  T007  2008      6   
29           UNSPECIFIED CULEX     62       N MANDELL AVE  T007  2008      6   
...                        ...    ...                 ...   ...   ...    ...   
249506  CULEX PIPIENS/RESTUANS     51    N MONT CLARE AVE  T223  2013      9   
249507  CULEX PIPIENS/RESTUANS     51    N MONT CLARE AVE  T223  2013      9   
249508           CULEX PIPIENS     51    N MONT CLARE AVE  T223  2013      9   
249509           CULEX PIPIENS     51    N MONT CLARE AVE  T223  2013      9   
249510  CULEX PIPIENS/RESTUANS     82       S KOSTNER AVE  T225  2013      9   
249511  CULEX PIPIENS/RESTUANS     82       S KOSTNER AVE  T225  2013      9   
249512  CULEX PIPIENS/RESTUANS     65           E 91ST PL  T227  2013      9   
249513  CULEX PIPIENS/RESTUANS     65           E 91ST PL  T227  2013      9   
249514           CULEX PIPIENS     65           E 91ST PL  T227  2013      9   
249515           CULEX PIPIENS     65           E 91ST PL  T227  2013      9   
249516  CULEX PIPIENS/RESTUANS     17        W ADDISON ST  T224  2013      9   
249517  CULEX PIPIENS/RESTUANS     17        W ADDISON ST  T224  2013      9   
249518  CULEX PIPIENS/RESTUANS     90     W GARFIELD BLVD  T226  2013      9   
249519  CULEX PIPIENS/RESTUANS     90     W GARFIELD BLVD  T226  2013      9   
249520  CULEX PIPIENS/RESTUANS     13       N LARAMIE AVE  T230  2013      9   
249521  CULEX PIPIENS/RESTUANS     13       N LARAMIE AVE  T230  2013      9   
249522           CULEX PIPIENS     13       N LARAMIE AVE  T230  2013      9   
249523           CULEX PIPIENS     13       N LARAMIE AVE  T230  2013      9   
249524  CULEX PIPIENS/R

In [9]:
#minority_upsampled = resample( minority_class, replace=True, n_samples=majority_class.shape[0], random_state=42)
#train_resampled = pd.concat([minority_upsampled,majority_class])
#train_resampled.wnvpresentd.value_counts()

In [10]:
#train_resampled.head()